<a href="https://colab.research.google.com/github/CH4nt014/Big-Data-21-22-Gruppo1/blob/main/importCsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [27]:
#Modifica la precisione nella visualizzazione delle cifre
pd.options.display.precision = 10

#Legge il csv e lo stampa
df_DatiSensori = pd.read_excel('Dati-gruppo1.xlsx')

#Mostra tutte le colonne
# pd.option_context('display.max_rows', df_DatiSensori.shape[0]+1)
df_DatiSensori

,postazione,Data,TRS_ppb,TRS_stato,VOC_ppm,VOC_stato,C6H6_ug/m3,C6H6_stato,H2S_ug/m3,H2S_stato,H2SJ_ug/m3,H2SJ_stato,PIDVOC_ppb,PIDVOC_stato
0,ATM05_01479,2021-04-12 00:00:00,2.0000000000,K,0.0,K,0.3000000119,K,0.0000000000,K,0.0000000000,K,44.5999984741,K
1,ATM05_01479,2021-04-12 00:05:00,0.0000000000,K,0.0,K,0.2000000030,K,0.2000000030,K,0.0000000000,K,44.0000000000,K
2,ATM05_01479,2021-04-12 00:10:00,0.0000000000,K,0.0,K,0.2000000030,K,1.7000000477,K,0.0000000000,K,44.2000007629,K
3,ATM05_01479,2021-04-12 00:15:00,11.8999996185,K,0.0,K,0.2000000030,K,1.8999999762,K,0.0000000000,K,44.4000015259,K
4,ATM05_01479,2021-04-12 00:20:00,26.1000003815,K,0.0,K,0.2000000030,K,2.5000000000,K,0.0000000000,K,40.7999992371,K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195547,ATM14_01486,2021-11-28 23:35:00,0.0000000000,K,0.0,K,0.6000000238,K,8.8999996185,K,0.3000000119,K,352.6000061035,K
195548,ATM14_01486,2021-11-28 23:40:00,0.0000000000,K,0.0,K,0.6000000238,K,8.3000001907,K,0.3000000119,K,350.6000061035,K
195549,ATM14_01486,2021-11-28 23:45:00,0.0000000000,K,0.0,K,0.6000000238,K,8.6999998093,K,0.3000000119,K,353.0000000000,K
195550,ATM14_01486,2021-11-28 23:50:00,0.0000000000,K,0.0,K,0.6000000238,K,9.1999998093,K,1.2000000477,K,352.7999877930,K


In [5]:
pd.set_option('display.max_rows', 100)
# query effettuata con un unico comando
# df_Benzene = df_DatiSensori.sort_values(by='C6H6_ug/m3', ascending =False)[['postazione', 'Data', 'C6H6_ug/m3']].head(100)

# creazione di un dataFrame ordinato in modo discendente per i valori di benzene
df_Benzene = df_DatiSensori.sort_values(by='C6H6_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di benzene
query_Benzene = df_Benzene[['postazione', 'Data', 'C6H6_ug/m3']].head(100)
query_Benzene

,postazione,Data,C6H6_ug/m3
88290,ATM10_01481,2021-07-10 13:30:00,10.3999996185
66647,ATM07_01480,2021-11-08 09:55:00,5.9000000954
106472,ATM10_01481,2021-11-20 16:40:00,3.0000000000
106465,ATM10_01481,2021-11-20 16:05:00,2.7999999523
88293,ATM10_01481,2021-07-10 13:45:00,2.5000000000
122287,ATM14_01486,2021-06-04 14:35:00,2.5000000000
106471,ATM10_01481,2021-11-20 16:35:00,2.0000000000
106473,ATM10_01481,2021-11-20 16:45:00,1.5000000000
122286,ATM14_01486,2021-06-04 14:30:00,1.3999999762
106476,ATM10_01481,2021-11-20 17:00:00,1.3999999762


In [24]:
# query effettuata con un unico comando
# df_H2S = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False)[['postazione', 'Data', 'H2S_ug/m3']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori di acido solfidrico del sensore H2S
df_H2S = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di acido solfidrico del sensore H2SJ
query_H2S = df_H2S[['postazione', 'Data', 'H2S_ug/m3', 'H2SJ_ug/m3']].head(100)
query_H2S

,postazione,Data,H2S_ug/m3,H2SJ_ug/m3
122287,ATM14_01486,2021-06-04 14:35:00,169.0000000000,0.4000000060
93529,ATM10_01481,2021-08-11 18:05:00,120.1999969482,1.1000000238
93527,ATM10_01481,2021-08-11 17:55:00,115.6999969482,21.6000003815
93528,ATM10_01481,2021-08-11 18:00:00,113.8000030518,1.2000000477
93526,ATM10_01481,2021-08-11 17:50:00,113.6999969482,6.1999998093
96411,ATM10_01481,2021-07-31 18:15:00,113.4000015259,1.1000000238
93514,ATM10_01481,2021-08-11 16:50:00,109.8000030518,1.1000000238
93518,ATM10_01481,2021-08-11 17:10:00,108.5999984741,1.2000000477
88041,ATM10_01481,2021-07-09 16:45:00,108.0999984741,0.0000000000
96412,ATM10_01481,2021-07-31 18:20:00,107.8000030518,1.1000000238


In [9]:
# query effettuata con un unico comando
# df_H2SJ = df_DatiSensori.sort_values(by='H2SJ_ug/m3', ascending =False)[['postazione', 'Data', 'H2SJ_ug/m3']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori di acido solfidrico del sensore H2SJ
df_H2SJ = df_DatiSensori.sort_values(by='H2SJ_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di acido solfidrico del sensore H2SJ
query_H2SJ = df_H2SJ[['postazione', 'Data', 'H2SJ_ug/m3']].head(100)
query_H2SJ

,postazione,Data,H2SJ_ug/m3
83989,ATM10_01481,2021-06-25 15:05:00,56.9000015259
10391,ATM05_01479,2021-06-22 01:55:00,56.9000015259
90592,ATM10_01481,2021-07-18 13:20:00,56.9000015259
23493,ATM05_01479,2021-07-30 13:45:00,56.7999992371
8785,ATM05_01479,2021-06-16 12:05:00,56.7999992371
23494,ATM05_01479,2021-07-30 13:50:00,56.7000007629
72086,ATM07_01480,2021-11-27 07:10:00,56.7000007629
83047,ATM10_01481,2021-06-22 08:35:00,56.7000007629
23495,ATM05_01479,2021-07-30 13:55:00,56.7000007629
84547,ATM10_01481,2021-06-27 13:35:00,56.5999984741


In [23]:
# df_Acido = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# df_Acido = df_DatiSensori.sort_values(['H2S_ug/m3','H2SJ_ug/m3'],ascending=[False,False]).head(100)

query_Acido = df_Acido[['postazione', 'Data', 'H2S_ug/m3', 'H2SJ_ug/m3']]
query_Acido

,postazione,Data,H2S_ug/m3,H2SJ_ug/m3
122287,ATM14_01486,2021-06-04 14:35:00,169.0000000000,0.4000000060
93529,ATM10_01481,2021-08-11 18:05:00,120.1999969482,1.1000000238
93527,ATM10_01481,2021-08-11 17:55:00,115.6999969482,21.6000003815
93528,ATM10_01481,2021-08-11 18:00:00,113.8000030518,1.2000000477
93526,ATM10_01481,2021-08-11 17:50:00,113.6999969482,6.1999998093
96411,ATM10_01481,2021-07-31 18:15:00,113.4000015259,1.1000000238
93514,ATM10_01481,2021-08-11 16:50:00,109.8000030518,1.1000000238
93518,ATM10_01481,2021-08-11 17:10:00,108.5999984741,1.2000000477
88041,ATM10_01481,2021-07-09 16:45:00,108.0999984741,0.0000000000
96412,ATM10_01481,2021-07-31 18:20:00,107.8000030518,1.1000000238


In [ ]:
# query effettuata con un unico comando
# df_VOC = df_DatiSensori.sort_values(by='VOC_ppm', ascending =True)[['postazione', 'Data', 'VOC_ppm']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori del sensore VOC
df_VOC = df_DatiSensori.sort_values(by='VOC_ppm', ascending =True).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 più bassi valori del sensore VOC
query_VOC = df_VOC[['postazione', 'Data', 'VOC_ppm']].head(100)
query_VOC

In [ ]:
# query effettuata con un unico comando
# df_PIDVOC = df_DatiSensori.sort_values(by='PIDVOC_ppb', ascending =True)[['postazione', 'Data', 'PIDVOC_ppb']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori del sensore PIDVOC
df_PIDVOC = df_DatiSensori.sort_values(by='PIDVOC_ppb', ascending =True).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 più bassi valori del sensore PIDVOC
query_PIDVOC = df_PIDVOC[['postazione', 'Data', 'PIDVOC_ppb']].head(100)
query_PIDVOC

In [ ]:
# query 4: dividere il dataset in gruppi di ore consecutive, sommare le ore e calcolare la media per ogni ora 
# e quella più bassa